In [56]:
from bs4 import BeautifulSoup
from urllib.parse import urlencode, urlunparse
from textgenrnn import textgenrnn
import requests
import pandas as pd

In [16]:
params = {'q': 'london', 'page_size': 100, 'pn': 1}
url = urlunparse(['http', 'www.zoopla.co.uk', 'to-rent/property/london/', '', urlencode(params), ''])
search_page = BeautifulSoup(requests.get(url).content, 'html5lib')
listings = search_page.find_all('li', attrs={'data-listing-id': True})
print(url)
print(len(listings))

http://www.zoopla.co.uk/to-rent/property/london/?q=london&page_size=100&pn=1
100


In [65]:
def parse_listing(listing):
    return {
        'listing_id': int(listing.attrs['data-listing-id']),
        'price': int(list(listing.find('a', class_='text-price').children)[0].replace('pcm', '').replace(',', '').replace(u'\xa3', '').strip()),
#         'beds': int(listing.find('span', class_='num-beds').get_text().strip()) if listing.find('span', class_='num-beds') is not None else None,
        'url': 'https://www.zoopla.co.uk/to-rent/details/' + listing.attrs['data-listing-id'],
    }

In [66]:
df = pd.DataFrame(map(parse_listing, listings))
df.head()

,listing_id,price,url
0,51470233,1257,https://www.zoopla.co.uk/to-rent/details/51470233
1,51470227,2167,https://www.zoopla.co.uk/to-rent/details/51470227
2,51470224,1800,https://www.zoopla.co.uk/to-rent/details/51470224
3,51470226,2100,https://www.zoopla.co.uk/to-rent/details/51470226
4,51470218,1755,https://www.zoopla.co.uk/to-rent/details/51470218


In [98]:
%%time
df['description'] = pd.Series([
    BeautifulSoup(requests.get(df.iloc[i]['url']).content, 'html5lib')
        .find('div', class_='dp-description__text')
        .get_text('\n')
    for i in range(len(df))
])

In [99]:
df.to_csv('zoopla100.csv', index=False)

In [100]:
print(df['description'][0])


                                        We are pleased to present this 1 bed property in Hackney.
It is £290 pw
Property features:
- cooker, fridge and more
- good sized property
- separate fitted kitchen
DSS with guarantor accepted for this property
The property is close to all local amenities, shops, restaurants etc. Bills are included with the rent price. One-month rent, one-month deposit and agency fees apply. For more information please give us a call.
                                    


In [106]:
with open("zoopla_descriptions100.txt", "w") as f:
    f.write('\n'.join(df['description'].values))

In [107]:
%%time
textgen = textgenrnn()
textgen.train_from_file("zoopla_descriptions100.txt", num_epochs=1)

687 texts collected.
Training on 66,025 character sequences.
Epoch 1/1
515/515 [==============================] - 93s 181ms/step - loss: 1.3305
####################
Temperature: 0.2
####################
                                                                                                                                                                                                                                                                                                           

                                                                                                                                                                                                                                                                                                           

                                                                                                                                                                                                   

In [108]:
textgen.generate()

The bedroom floor from the property flooring and offers a property funny and the property flooring and amazie this property walking kitchen with a station of the first reception room. Market in the property flooring within situated convinity with a comprises of the reception furnished and fitted re

